In [1]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, InputLayer
from keras.optimizers import RMSprop

import tensorflow as tf

Using TensorFlow backend.


In [0]:
batch_size = 128
num_classes = 10
epochs = 5

**Load and Split data**

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [9]:
print('Shape of x_train : {}'.format(x_train.shape))
print('Shape of x_test : {}'.format(x_test.shape))
print('Shape of y_train : {}'.format(y_train.shape))
print('Shape of y_test : {}'.format(y_test.shape))

Shape of x_train : (60000, 28, 28)
Shape of x_test : (10000, 28, 28)
Shape of y_train : (60000,)
Shape of y_test : (10000,)


**Reshape data set and Chage type to float3 **

In [20]:
x_train.dtype

dtype('uint8')

In [0]:
x_train = x_train.reshape(60000, 784).astype('float32')
x_test = x_test.reshape(10000, 784).astype('float32')

In [24]:
x_train.dtype

dtype('float32')

**Normalization**

In [25]:
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')

60000 train samples


**Label 을 One-hot vector로 변환**
- 총 10개의 값 중 label만 1, 나머지 0

In [27]:
# 첫번째 이미지 데이터의 label 은 '5'
y_train[0]

5

In [0]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [29]:
#One-hot encoding 후
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

**Keras 모델 생성**

In [0]:
model = Sequential()

model.add(Dense(512, activation='relu', input_dim=784))
# model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

In [72]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

In [74]:
history = model.fit(x_train, y_train,
                   batch_size = batch_size,
                   epochs = epochs,
                   verbose =1,
                   validation_data = (x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss', score[0])
print('Test accuracy: ', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 53us/step - loss: 0.8494 - acc: 0.7410 - val_loss: 0.4217 - val_acc: 0.8685
Epoch 2/5
60000/60000 [==============================] - 3s 51us/step - loss: 0.3599 - acc: 0.8937 - val_loss: 0.3023 - val_acc: 0.9111
Epoch 3/5
60000/60000 [==============================] - 3s 51us/step - loss: 0.2884 - acc: 0.9145 - val_loss: 0.2502 - val_acc: 0.9254
Epoch 4/5
60000/60000 [==============================] - 3s 51us/step - loss: 0.2419 - acc: 0.9283 - val_loss: 0.2387 - val_acc: 0.9323
Epoch 5/5
60000/60000 [==============================] - 3s 50us/step - loss: 0.2034 - acc: 0.9392 - val_loss: 0.1768 - val_acc: 0.9453
Test loss 0.17678991677165032
Test accuracy:  0.9453


In [0]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
  from tensorflow.python.framework.graph_util import convert_variables_to_constants
  graph = session.graph
  with graph.as_default():
    freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
    output_names = output_names or []
    output_names += [v.op.name for v in tf.global_variables()]
    input_graph_def = graph.as_graph_def()
    if clear_devices:
      for node in input_graph_def.node:
        node.device=""
    frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                 output_names, freeze_var_names)
    return frozen_graph

In [76]:
from keras import backend as K
from keras import backend as K


custom_input_tensor = tf.placeholder(tf.float32, shape=(1, 784))
output_tensor = model(custom_input_tensor)
output_names = [node.op.name for node in model.outputs]

frozen_graph = freeze_session(K.get_session(), output_names=[output_tensor.op.name])


INFO:tensorflow:Froze 32 variables.
INFO:tensorflow:Converted 32 variables to const ops.


In [0]:
tflite_model = tf.contrib.lite.toco_convert(frozen_graph, 
                                            [custom_input_tensor],
                                            [output_tensor])
open("mnist_mlp.tflite", "wb").write(tflite_model)
files.download('mnist_mlp.tflite')